# Diferenciación automática

**Nota**: Los ejercicios de este notebook se iniciarán en clase y corresponden a la Tarea 2.

## Definición de nuevos tipos

Una de las partes atractivas de Julia es que permite definir nuevos tipos (o estructuras) que en algún sentido reflejan la abstracción de las componentes de un problema concreto. Para esto se utilizan las instrucciones `struct` y `mutable struct`.

Para ilustrar esto usaremos como ejemplo la *diferenciación automática*, también llamada *diferenciación algorítmica*, usando los [*números duales*](https://en.wikipedia.org/wiki/Automatic_differentiation#Automatic_differentiation_using_dual_numbers).

La idea de los números duales es, en algún sentido, parecida a la de los números complejos. En el caso de los números complejos, escribimos $z = x +i y$, donde el número especial $i$ se define a partir de $i^2=-1$. Con esta definición, podemos extender las operaciones aritméticas al igual que las funciones elementales. Vale la pena recordar que $\mathbb{C}$ es *isomorfo* a $\mathbb{R}^2$, esto es, hay un mapeo biyectivo de $ z \leftrightarrow (x, y)$.

De manera similar, definimos los números duales como un par ordenado, que escribiremos $ x + x' \varepsilon$, donde $\varepsilon$ tiene la propiedad $\varepsilon^2 = 0$. De igual manera que en el caso de los números complejos, los números duales puedden ser representados por la dupla ordenada $\{x,x'\}$, lo que sugiere la manera en que los definiremos en Julia, más adelante.

A partir de la definición, es fácil convencerse que se cumple:

\begin{eqnarray*}
(x + x' \varepsilon ) + (y + y' \varepsilon) & = & (x+y) + (x'+y')\varepsilon,\\
(x + x' \varepsilon ) \cdot (y + y' \varepsilon) & = & xy + (x'y+xy')\varepsilon.\\
\end{eqnarray*}

A partir de estas relaciones, podemos evaluar polinomios en dichas variables, obteniendo

\begin{eqnarray}
P_n(x+x'\varepsilon) & = & p_0 +p_1(x+x'\varepsilon) + p_2(x+x'\varepsilon)^2+\dots+p_n(x+x'\varepsilon)^n\\
& = & p_0 + p_1(x+x'\varepsilon) + p_2(x^2+2xx'\varepsilon) + \dots +p_n(x^n+nx^{n-1}x'\varepsilon)\\
& = & (p_0 + p_1x + p_2x^2\dots p_nx^n) + (p_1+2p_2x+\dots np_nx^{n-1})x'\varepsilon\\
& = & P_n(x) + P_n'(x)x'\varepsilon,
\end{eqnarray}

donde $P_n'(x)$ es la (primer) derivada de $P_n(x)$ evaluada en $x$. Esto es, la segunda componente corresponde a la derivada del polinomio, en $x$, incluyendo la regla de la cadena. Como veremos, esta propiedad la explotaremos para calcular *numéricamente* derivadas.

En particular, explotando la propiedad anterior, el polinomio (o función) constante lo identificaremos con la dupla $\{c,0\}$, y a la *variable independiente* $x$ en $x_0$ la escribiremos con la dupla $\{x_0, 1\}$.

A partir de lo anterior, definiremos la siguiente *estructura*, que identificaremos con cada una de los miembros de las duplas descritas anteriormente. Lo siguiente define a la estructura `Dual` como una estructura inmutable.

In [1]:
"""
    Dual

Definición de los números duales. Los campos internos son
    x  :: Float64   # valor de la función
    x′ :: Float64   # valor de su derivada

"""
struct Dual
    x  :: Float64
    x′ :: Float64
end

Dual

In [12]:
a = Dual(1.2,1)

Dual(1.2, 1.0)

In [5]:
isimmutable( a )

true

Para acceder individualmente a los campos del `Dual`, usamos los nombres de los campos que usamos al definir la estructura:

In [13]:
a.x

1.2

In [14]:
a.x′

1.0

Ahora definiremos un método apropiado para obtener el `Dual` de una constante numérica:

In [6]:
Dual(c::Real) = Dual(c, 0.0)

Dual

In [7]:
Dual(pi)

Dual(3.141592653589793, 0.0)

La función `dual(x0)` construirá el `Dual` de la variable independiente $x$ en $x_0$:

In [8]:
dual(x0::Real) = Dual(x0, 1.0)

dual (generic function with 1 method)

## Ampliando la funcionalidad de `Dual`

Ahora ampliaremos la funcionalidad de los `Dual`es. Empezaremos por extender las operaciones aritméticas. En lo siguiente, escribiremos a los duales como $\vec{u} = \{u_0, u_0'\}$.

Las operaciones aritméticas entonces quedan definidas como:

\begin{eqnarray}
    \vec{u} \pm \vec{w} &=& \big\{ u_0 \pm w_0, \, u'_0\pm w'_0 \big\},\\
    \vec{u} \times \vec{w} &=& \big\{ u_0 \cdot w_0,\, u_0 w'_0 +  w_0 u'_0 \big\},\\
    \frac{\vec{u}}{\vec{w}} &=& \big\{ \frac{u_0}{w_0},\, \frac{ u'_0 - (u_0/w_0)w'_0}{w_0}\big\},\\
    {\vec{u}}^\alpha &=& \big\{ u_0^\alpha,\, \alpha u_0^{\alpha-1} u'_0 \big\}.\\
\end{eqnarray}    


Para poder *extender* los operadores `+`, `-`, `*`, `/` y `^`, primero debemos importarlas:

In [10]:
# Importamos la definición de los siguientes operadores
import Base: +, -, *, /, ^

Y ahora las extendemos a conveniencia:

In [23]:
+(u::Dual, v::Dual) = Dual( u.x + v.x, u.x′ + v.x′)

-(u::Dual, v::Dual) = Dual( u.x - v.x, u.x′ - v.x′)

- (generic function with 195 methods)

In [24]:
*(u::Dual, v::Dual) = Dual( u.x * v.x, u.x * v.x′ + u.x′ * v.x)

* (generic function with 183 methods)

In [25]:
function /(u::Dual, v::Dual)
    y = u.x / v.x
    Dual( y, (u.x′ - y * v.x′)/v.x )
end

/ (generic function with 74 methods)

In [26]:
function ^(a::Dual, n::Int)
    y = a.x^(n-1)
    Dual(a.x * y, n*y*a.x′)
end

^ (generic function with 53 methods)

In [29]:
Dual(1,2) - Dual(1,2)

Dual(0.0, 0.0)

Si bien estas operaciones funcionan, los siguientes casos no lo hacen:

In [30]:
3.2 + Dual(1,2)

LoadError: [91mMethodError: no method matching +(::Float64, ::Dual)[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  +(::Float64, [91m::Float64[39m) at float.jl:375
  +([91m::Dual[39m, ::Dual) at In[23]:1
  ...[39m

Como se indica en el mensaje de error, hace falta definir las operaciones que involucren números (`Real`) y `Dual`es; noten que definimos ambas posibilidades, lo que permite usar `+` con un `Real` y un `Dual` en cualquier orden.

In [32]:
+(a::Real, u::Dual) = Dual( a + u.x, u.x′)
+(u::Dual, a::Real) = Dual( a + u.x, u.x′)

+ (generic function with 183 methods)

### Ejercicio 1

Extiendan los operadores `+`, `-`, `*` y `/` para que funcionen cuando una de las dos entradas es un `Dual` y la otra es un `Real` (número cualquiera), sin importar el orden.

## Algunos ejemplos del uso de `Dual`

En lo que sigue, verificaremos dos cosas:

- la implementación de las operaciones aritméticas es la correcta,

- cómo obtener la derivada de polinomios o funciones racionales.


Para el primer punto, usaremos una paquetería que sirve para *definir* tests. La idea es checar la funcionalidad usando ejemplos concretos verificables.

In [20]:
# Esto carga la paquetería Base.Test
using Base.Test

In [21]:
# Esto checa que Dual(1,2)+Dual(-1,-2) es idéntico a Dual(0,0)

In [27]:
@test Dual(1,2)+Dual(-1,-2) == Dual(0.0)

Test Passed

### Ejercicio 2

Construyan 2 tests para cada una de las operaciones aritméticas definidas para los `Dual`es, incluyendo las que definieron en el ejercicio 1.

### Ejercicio 3

Sobrecarguen la función `show` (que está en Base) para que cuando se impriman los duales aparezcan con la notación $\varepsilon$ que usamos al principio de este notebook, por ejemplo, el resultado de `Dual(1,2)` debe ser parecido a `1.0 + 2.0 ε`.

En cuanto a la segunda parte, consideraremos el siguiente caso concreto: $f(x) = x^2-1$, y queremos averiguar el valor de la derivada en $x=3$.

Primero, definiremos la función $f(x)$, y también el `Dual` `x`, que corresponde a la variable independiente en `x=3`.

In [36]:
f(x) = x^2 + 1

f (generic function with 1 method)

In [37]:
x = dual(3)

Dual(3.0, 1.0)

Ahora, sustituiremos `x` definida arriba en `f`:

In [38]:
f(x)

Dual(10.0, 6.0)

Es claro que la primer componente corresponde al valor de la función evaluada en $x_0$, $f(3)=10$; la segunda componente es la derivada evaluada en el mismo punto, es decir, `f'(3) = 2*3=6`.

### Ejercicio 4

Obtengan la derivada de 
$$g(x) = \frac{3x^2-8x+5}{7x^3-1}$$
en $x_0=1$.

### Ejercicio 5

- Recordando la regla de la cadena(!!!), extiendan el usar los `Dual` a las funciones `sqrt`, `exp`, `log`, `sin`, `cos`, `sinh` y `cosh`. 

- Muestren que las cosas dan los resultados que esperan usando pruebas como hicieron en el ejercicio 2.

- Calculen la derivada de $h(x) = \sin\Big(x^3 - \frac{2}{x^6}\Big)$ en $x_0 = 2$. ¿Qué tan preciso es el resultado?(Pueden usar cualquier otra manera de obtener el resultado correcto, sólo tienen que ser claros en la explicación.)

- Dibujen, para $x_0 \in [1,5]$ la función $h'(x)$.

### Ejercicio 6

Implementen el método de Newton para una función arbitraria $f(x)$ en una variable, explotando el uso de los `Dual`. 

- Obtengan, usando su función para el método de Newton, las raices del polinomio de Wilkinson 

$$
W_{6}(x) = (x-1)(x-2)(x-3)(x-4)(x-5)(x-6),
$$ 

usando como iterados iniciales $x_0=2.2$ y $x_0=2.45$.
    
- Hagan lo mismo que en el inciso anterior para $g(x) = x \sin(2x)$, con $x_0=0.7$.

### Ejercicio 7

Argumenten qué podrían hacer para extender la idea de los `Dual` y calcular derivadas aún más altas. Como caso concreto, piensen en querer obtener la derivada 18 de funciones como las que hemos usado arriba

## Módulos en Julia

En Julia, uno puede definir módulos (`module`), que son una serie de estructuras y tipos que sirven para hacer cuestiones concretas, como por ejemplo, implementar la diferenciación automática. Los módulos son muy útiles porque uno los puede almacenar en archivos que son llamados cuando se requiere. Lo que hemos hecho en este notebook, eventualmente será reutilizado, así que vale la pena que experimenten la cración de módulos.
